In [12]:
# Load once and reuse across multiple derivation calls
import sqlite3

# Change this to your actual DB path
conn = sqlite3.connect("db_numprop-4_nestlim-100.db")
cursor = conn.cursor()

# Load all rows from the 'data' table
cursor.execute("SELECT * FROM data")
all_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]


In [49]:
import sqlite3
import random
import networkx as nx
import matplotlib.pyplot as plt

def extract_grammar_from_data_row(row, columns):
    """
    Build grammar from a row in the 'data' table.
    Each rule returns a nested tuple of the form:
      - Binary: (operator_name, 'X', 'X')
      - Unary:  (operator_name, 'X')
    """
    operator_mapping = {
         "A": {"name": "and", "arity": 2},
         "O": {"name": "or", "arity": 2},
         "C": {"name": "conditional", "arity": 2},
         "NC": {"name": "not_conditional", "arity": 2},
         "B": {"name": "biconditional", "arity": 2},
         "X": {"name": "not_biconditional", "arity": 2},
         "NA": {"name": "not_and", "arity": 2},
         "NOR": {"name": "not_or", "arity": 2},
         "N": {"name": "not", "arity": 1},
    }

    def make_binary_rule(name):
        return lambda X: (name, X, X)

    def make_unary_rule(name):
        return lambda X: (name, X)

    grammar = {}
    for op, props in operator_mapping.items():
        if op in columns:
            idx = columns.index(op)
            if row[idx] == 1:
                if props["arity"] == 1:
                    grammar[op] = make_unary_rule(props["name"])
                else:
                    grammar[op] = make_binary_rule(props["name"])
    return grammar


def expand_all_X(expr, grammar):
    """
    Recursively finds the leftmost 'X' in a nested tuple structure and replaces it
    with each possible grammar rule or terminal symbol.
    """
    if expr == "X":
        # Base case: single 'X' to replace
        expansions = []

        for rule in grammar.values():
            expansions.append(rule("X"))

        for terminal in ["p", "q", "r", "s"]:
            expansions.append(terminal)

        return expansions

    elif isinstance(expr, tuple):
        # Recursive case: traverse the structure to find the leftmost 'X'
        for i, sub in enumerate(expr):
            sub_expansions = expand_all_X(sub, grammar)
            if sub_expansions:
                # Replace the first expandable part and break
                results = []
                for new_sub in sub_expansions:
                    new_expr = list(expr)
                    new_expr[i] = new_sub
                    results.append(tuple(new_expr))
                return results
    return []


def run_derivation_for_row(row_idx, row, columns):
    """
    Expands from 'X' using grammar derived from the row.
    At each step:
      - current_options is overwritten to contain only indexed expansions.
      - Simulated times are used internally.
      - Prints everything for debugging.
    """
    print(f"Using row {row_idx}: {row}")
    grammar = extract_grammar_from_data_row(row, columns)

    start_expr = "X"
    current = start_expr

    current_options = {}

    while True:
        expansions = expand_all_X(current, grammar)
        if not expansions:
            break

        # Overwrite the options dictionary with numbered entries
        current_options.clear()
        for i, exp in enumerate(expansions):
            current_options[i] = exp

        # Simulated evaluation — in final implementation, your network picks from `current_options`
        simulated_times = {i: random.uniform(0, 10) for i in current_options}

        print(f"\nCurrent expression: {current}")
        print(f"Options dict:\n  {current_options}")

        # Simulate network picking the best (lowest simulated time)
        best_index = min(simulated_times, key=simulated_times.get)
        best_exp = current_options[best_index]
        print(f"  Selected: {best_exp}")

        current = best_exp  # Move to the next node

    print(f"\n[Row {row_idx}] Grammar used: {list(grammar.keys())}")
    return current_options


# Plotting function for testing.
def plot_derivation_tree(G, title="Derivation Tree"):
    plt.figure(figsize=(10, 6))
    pos = nx.spring_layout(G, seed=42)
    nx.draw(G, pos, with_labels=True, node_size=1500, node_color="lightyellow", font_size=10, arrows=True)
    plt.title(title)
    plt.show()

def main_from_loaded_data(all_rows, column_names, row_index=0):
    if row_index >= len(all_rows):
        print(f"Row {row_index} out of range.")
        return
    row = all_rows[row_index]
    G = run_derivation_for_row(row_index, row, column_names)
    # plot_derivation_tree(G, f"Derivation from row {row_index}") plotting


if __name__ == "__main__":
    # Change the DB path and row_index to test other configurations
    main_from_loaded_data(all_rows, column_names, row_index=100000)
    

Using row 100000: (1, 0, 1, 0, 0, 0, 0, 0, 0, 28940, 17, 'O(N(O(p,O(q,N(r)))),N(O(N(p),O(N(O(q,N(O(r,s)))),N(O(N(r),N(s)))))))')

Current expression: X
Options dict:
  {0: ('or', 'X', 'X'), 1: ('not', 'X'), 2: 'p', 3: 'q', 4: 'r', 5: 's'}
  Selected: ('or', 'X', 'X')

Current expression: ('or', 'X', 'X')
Options dict:
  {0: ('or', ('or', 'X', 'X'), 'X'), 1: ('or', ('not', 'X'), 'X'), 2: ('or', 'p', 'X'), 3: ('or', 'q', 'X'), 4: ('or', 'r', 'X'), 5: ('or', 's', 'X')}
  Selected: ('or', ('not', 'X'), 'X')

Current expression: ('or', ('not', 'X'), 'X')
Options dict:
  {0: ('or', ('not', ('or', 'X', 'X')), 'X'), 1: ('or', ('not', ('not', 'X')), 'X'), 2: ('or', ('not', 'p'), 'X'), 3: ('or', ('not', 'q'), 'X'), 4: ('or', ('not', 'r'), 'X'), 5: ('or', ('not', 's'), 'X')}
  Selected: ('or', ('not', 's'), 'X')

Current expression: ('or', ('not', 's'), 'X')
Options dict:
  {0: ('or', ('not', 's'), ('or', 'X', 'X')), 1: ('or', ('not', 's'), ('not', 'X')), 2: ('or', ('not', 's'), 'p'), 3: ('or', (

In [ ]:
db_numprop-4_nestlim-100.db